In [ ]:
pip install Opencv-python

In [ ]:
import cv2
import numpy as np

# 1. Capture Video
cap = cv2.VideoCapture(0)  # Or a video file

# 2. Background Subtraction (Example - for moving crowds)
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

# 3. Main Loop
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 4. Preprocess Frame (e.g., resize, convert to grayscale)
    frame = cv2.resize(frame, (640, 480))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 5. Apply Background Subtraction
    fg_mask = bg_subtractor.apply(gray)

    # 6.  Basic Blob Detection (Example - for counting moving regions)
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    num_people = len(contours)  #  Crude approximation

    # 7.  (Conceptual) Density Map Generation (Replace with a proper method)
    density_map = np.zeros_like(gray, dtype=np.float32)
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        #  Instead of a single point,  you'd spread the "density"
        #  around the location of the person, perhaps with a Gaussian.
        cv2.circle(density_map, (x + w // 2, y + h // 2), 5, 20, -1)  # Simplified

    # Normalize and convert to a displayable format
    density_map = cv2.normalize(density_map, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    colored_density_map = cv2.applyColorMap(density_map, cv2.COLORMAP_JET)


    # 8.  Visualization (Basic - extend this for creative output)
    cv2.putText(frame, f"People Count: {num_people}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    cv2.imshow("Frame", frame)
    cv2.imshow("Foreground Mask", fg_mask)
    cv2.imshow("Density Map", colored_density_map) # Show the density map

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
